<a href="https://colab.research.google.com/github/amiya100/Molecular-Property-Prediction/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 14.1 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
from rdkit.Chem import AllChem
from rdkit import Chem
from collections import defaultdict


def load_data_long(dataset, device):
    label_len = 12

    pair_list = ['Br', 'Cl', 'Si', 'Na', 'Ca', 'Ge', 'Cu', 'Au', 'Sn', 'Tb', 'Pt', 'Re', 'Ru', 'Bi', 'Li', 'Fe', 'Sb', 'Hg', 'Pb', 'Se', 'Ag', 'He', 'Cr', 'Pd', 'Ga', 'Mg', 'Ni', 'Ir', 'Rh', 'Te', 'Ti', 'Al', 'Zr', 'Tl', 'Zn', 'Be', 'Ne', 'Ar', 'Mn', 'Co', 'As', 'Kr', 'Rb', 'Sr', 'Nb', 'Mo', 'Tc', 'Cd', 'In', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Os', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

    mole_dict = {
        1: "H", 2: "He", 3: "Li", 4: "Be", 5: "B", 6: "C", 7: "N", 8: "O", 9: "F", 10: "Ne",
        11: "Na", 12: "Mg", 13: "Al", 14: "Si", 15: "P", 16: "S", 17: "Cl", 18: "Ar", 19: "K", 20: "Ca",
        21: "Sc", 22: "Ti", 23: "V", 24: "Cr", 25: "Mn", 26: "Fe", 27: "Co", 28: "Ni", 29: "Cu", 30: "Zn",
        31: "Ga", 32: "Ge", 33: "As", 34: "Se", 35: "Br", 36: "Kr", 37: "Rb", 38: "Sr", 39: "Y", 40: "Zr",
        41: "Nb", 42: "Mo", 43: "Tc", 44: "Ru", 45: "Rh", 46: "Pd", 47: "Ag", 48: "Cd", 49: "In", 50: "Sn",
        51: "Sb", 52: "Te", 53: "I", 54: "Xe", 55: "Cs", 56: "Ba", 57: "La", 58: "Ce", 59: "Pr", 60: "Nd",
        61: "Pm", 62: "Sm", 63: "Eu", 64: "Gd", 65: "Tb", 66: "Dy", 67: "Ho", 68: "Er", 69: "Tm", 70: "Yb",
        71: "Lu", 72: "Hf", 73: "Ta", 74: "W", 75: "Re", 76: "Os", 77: "Ir", 78: "Pt", 79: "Au", 80: "Hg",
        81: "Tl", 82: "Pb", 83: "Bi", 84: "Po", 85: "At", 86: "Rn", 87: "Fr", 88: "Ra", 89: "Ac", 90: "Th",
        91: "Pa", 92: "U", 93: "Np", 94: "Pu", 95: "Am", 96: "Cm", 97: "Bk", 98: "Cf", 99: "Es", 100: "Fm",
        101: "Md", 102: "No", 103: "Lr", 104: "Rf", 105: "Db", 106: "Sg", 107: "Bh", 108: "Hs", 109: "Mt",
        110: "Ds", 111: "Rg", 112: "Cn", 113: "Nh", 114: "Fl", 115: "Mc", 116: "Lv", 117: "Ts", 118: "Og"
    }

    data_file = f"tox21_train.txt"
    file = open(data_file, "r")
    node_types = set()
    # label_types = set()
    tr_len = 0
    for line in file:
        tr_len += 1
        smiles = line.split("\t")[1]
        i = 0
        s = []
        while i < len(smiles):
            if i < len(smiles)-1 and (smiles[i] + smiles[i+1]) in pair_list:
                s.append(smiles[i] + smiles[i+1])
                i += 2
            else:
                s.append(smiles[i].upper())
                i += 1
        node_types |= set(s)
    print(node_types)
    file.close()

    te_len = 0
    data_file = f"tox21_test.txt"
    file = open(data_file, "r")
    for line in file:
        te_len += 1
        smiles = line.split("\t")[1]
        # label = line.split("\t")[2][:-1]
        i = 0
        s = []
        while i < len(smiles):
            if i < len(smiles)-1 and (smiles[i] + smiles[i+1]) in pair_list:
                s.append(smiles[i] + smiles[i+1])
                i += 2
            else:
                s.append(smiles[i].upper())
                i += 1
        node_types |= set(s)
        # label_types.add(label)
    file.close()

    print(tr_len)
    print(te_len)

    node2index = {n: i for i, n in enumerate(node_types)}
    # label2index = {l: i for i, l in enumerate(label_types)}

    print(node2index)
    # print(label2index)

    data_file = f"{dataset}_train.txt"
    file = open(data_file, "r")
    train_adjlists = []
    train_features = []
    train_sequence = []
    train_labels = torch.zeros(tr_len, label_len)
    for line in file:
        smiles = line.split("\t")[1]
        col_i = 0
        for l_l in range(2, label_len+1):
            train_labels[len(train_adjlists), col_i] = int(line.split("\t")[l_l])
            col_i += 1
        train_labels[len(train_adjlists), label_len-1] = int(line.split("\t")[-1][:-1])
        mol = AllChem.MolFromSmiles(smiles)
        graph_nodes = []
        for atom in mol.GetAtoms():
            graph_nodes.append(mole_dict[atom.GetAtomicNum()])
        i = 0
        s = 0
        while i < len(smiles):
            if i < len(smiles)-1 and (smiles[i] + smiles[i+1]) in pair_list:
                i += 2
            else:
                i += 1
            s += 1
        feature = torch.zeros(s, len(node_types))
        map = {}
        se_num = 0
        gr_num = 0
        i = 0
        smiles_seq = []
        while i < len(smiles):
            this_str = smiles[i]
            if i < len(smiles)-1 and (smiles[i] + smiles[i+1]) in pair_list:
                this_str = smiles[i] + smiles[i+1]
                i += 2
            else:
                this_str = this_str.upper()
                i += 1
            smiles_seq.append(node2index[this_str])
            if this_str in graph_nodes and this_str == mole_dict[mol.GetAtoms()[gr_num].GetAtomicNum()]:
                map[gr_num] = se_num
                gr_num += 1
            feature[se_num, node2index[this_str]] = 1
            se_num += 1
        print()
        adj_list = defaultdict(list)
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            typ = bond.GetBondType()
            adj_list[map[i]].append(map[j])
            adj_list[map[j]].append(map[i])
            if typ == Chem.rdchem.BondType.DOUBLE:
                adj_list[map[i]].append(map[j])
                adj_list[map[j]].append(map[i])
            elif typ == Chem.rdchem.BondType.TRIPLE:
                adj_list[map[i]].append(map[j])
                adj_list[map[j]].append(map[i])
                adj_list[map[i]].append(map[j])
                adj_list[map[j]].append(map[i])
        train_adjlists.append(adj_list)
        train_features.append(torch.FloatTensor(feature).to("cpu"))
        train_sequence.append(torch.tensor(smiles_seq))
    file.close()
    print(map)

    data_file = f"tox21_test.txt"
    file = open(data_file, "r")
    test_adjlists = []
    test_features = []
    test_sequence = []
    test_labels = np.zeros((te_len, label_len))
    for line in file:
        smiles = line.split("\t")[1]
        # print(smiles)
        col_i = 0
        for l_l in range(2, label_len+1):
            test_labels[len(test_adjlists), col_i] = int(line.split("\t")[l_l])
            col_i += 1
        test_labels[len(test_adjlists), label_len-1] = int(line.split("\t")[-1][:-1])
        mol = AllChem.MolFromSmiles(smiles)
        graph_nodes = []
        for atom in mol.GetAtoms():
            graph_nodes.append(mole_dict[atom.GetAtomicNum()])
        # print(graph_nodes)
        i = 0
        s = 0
        while i < len(smiles):
            if i < len(smiles)-1 and (smiles[i] + smiles[i+1]) in pair_list:
                i += 2
            else:
                i += 1
            s += 1

        feature = torch.zeros(s, len(node_types))

        map = {}
        se_num = 0
        gr_num = 0
        i = 0
        smiles_seq = []
        while i < len(smiles):
            this_str = smiles[i]
            if i < len(smiles)-1 and (smiles[i] + smiles[i+1]) in pair_list:
                this_str = smiles[i] + smiles[i+1]
                i += 2
            else:
                this_str = this_str.upper()
                i += 1
            smiles_seq.append(node2index[this_str])
            if this_str in graph_nodes and this_str == mole_dict[mol.GetAtoms()[gr_num].GetAtomicNum()]:
                map[gr_num] = se_num
                gr_num += 1
            feature[se_num, node2index[this_str]] = 1
            se_num += 1

        adj_list = defaultdict(list)
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            # print(i,j)
            typ = bond.GetBondType()
            adj_list[map[i]].append(map[j])
            adj_list[map[j]].append(map[i])
            if typ == Chem.rdchem.BondType.DOUBLE:
                adj_list[map[i]].append(map[j])
                adj_list[map[j]].append(map[i])
            elif typ == Chem.rdchem.BondType.TRIPLE:
                adj_list[map[i]].append(map[j])
                adj_list[map[j]].append(map[i])
                adj_list[map[i]].append(map[j])
                adj_list[map[j]].append(map[i])

        # test_labels[len(test_adjlists)] = int(label2index[label])
        # test_labels[len(test_adjlists)] = int(label)
        test_adjlists.append(adj_list)
        test_features.append(torch.FloatTensor(feature).to('cpu'))
        test_sequence.append(torch.tensor(smiles_seq))
    file.close()

    train_data = {}
    train_data['adj_lists'] = train_adjlists
    train_data['features'] = train_features
    train_data['sequence'] = train_sequence

    test_data = {}
    test_data['adj_lists'] = test_adjlists
    test_data['features'] = test_features
    test_data['sequence'] = test_sequence

    return train_data, train_labels, test_data, test_labels

In [ ]:
train_data, train_labels, test_data, test_labels=load_data_long("tox21", device="cuda:0" if torch.cuda.is_available() else "cpu")
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

{'2', 'Na', 'Zr', 'Br', 'Yb', '\\', 'Fe', 'C', '1', '[', 'Tl', '+', 'Se', 'I', 'Si', 'Be', '/', 'O', 'Pt', 'Sn', 'Ti', 'Sr', 'F', 'Hg', 'Li', '.', 'N', 'Nd', 'Ag', '-', 'Cd', '4', ')', 'Cu', 'S', 'Pb', 'In', '=', 'Cr', 'Ba', 'Zn', ']', 'Cl', '@', 'Co', 'Ge', '8', 'Ca', 'Mg', 'Bi', 'Ni', '6', 'K', 'Dy', '7', 'Gd', 'Au', 'Mn', 'As', 'B', 'H', '3', 'Sb', 'Pd', '5', 'P', '#', '(', 'Al'}
6264
784
{'2': 0, 'Na': 1, 'Zr': 2, 'Br': 3, 'Yb': 4, '\\': 5, 'Fe': 6, 'C': 7, '1': 8, '[': 9, 'Tl': 10, '+': 11, 'Se': 12, 'I': 13, 'Si': 14, 'Be': 15, '/': 16, 'O': 17, 'Pt': 18, 'Sn': 19, 'Ti': 20, 'Sr': 21, 'F': 22, 'Hg': 23, 'Li': 24, '.': 25, 'N': 26, 'Nd': 27, 'Ag': 28, '-': 29, 'Cd': 30, '4': 31, ')': 32, 'Cu': 33, 'S': 34, 'Pb': 35, 'In': 36, '9': 37, '=': 38, 'Cr': 39, 'Ba': 40, 'Zn': 41, ']': 42, 'Cl': 43, 'Mo': 44, '@': 45, 'Co': 46, 'Ge': 47, '8': 48, 'Ca': 49, 'Mg': 50, 'Bi': 51, 'Ni': 52, '6': 53, 'K': 54, 'Dy': 55, '7': 56, 'Gd': 57, 'Au': 58, 'Mn': 59, 'As': 60, 'B': 61, 'H': 62, '3': 63, 

[10:09:11] WARNING: not removing hydrogen atom without neighbors


Streaming output truncated to the last 5000 lines.





















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































